In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`


In [2]:
(q, N, L) = (3, 5, 4)

(3, 5, 4)

In [3]:
Q = 2N+4

14

In [4]:
T = Float32

Float32

In [5]:
using CUDA

In [6]:
f = zeros(T, 2N+4, L)
g = zeros(T, 2N+4, 2N+4, L-1)
JP = zeros(T, 2N+4, 2N+4)
tmpJP = zeros(T, 2N+4, 2N+4)
tmpf = zeros(T, 2N+4)
incf = zeros(T, 2N+4)
tmpg = zeros(T, 2N+4, 2N+4)
incg = zeros(T, 2N+4, 2N+4)
tmpdiagJP = zeros(T, 2N+4)
diagJP = zeros(T, 2N+4);

In [7]:
cond, J = BpAlignGpu.crea_instance_J_cond(Q, L);

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


In [9]:
conditional = zeros(T, Q, Q, L, L)
elts_J = zeros(T, Q, Q, L, L)
conditional .= cond
elts_J .= J;

In [8]:
f2 = BpAlignGpu.test_update_f(cond, J)

14×4 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  -46.2937  -54.01    0.0
 0.0  -62.3522  -65.769   0.0
 0.0  -54.8981  -49.9944  0.0
 0.0  -61.0016  -44.7134  0.0
 0.0  -42.8678  -36.2393  0.0
 0.0  -57.5786  -48.2782  0.0
 0.0  -54.3563  -62.9777  0.0
 0.0  -47.5234  -43.2405  0.0
 0.0  -56.7916  -38.8615  0.0
 0.0  -43.445   -46.7103  0.0
 0.0  -53.8189  -62.0427  0.0
 0.0  -68.0537  -43.6257  0.0
 0.0  -28.0959  -55.0965  0.0
 0.0  -61.879   -51.7773  0.0

In [11]:
BpAlignGpu.update_f!(f, JP, tmpJP, tmpf, incf, conditional, elts_J)
f

14×4 Matrix{Float32}:
 0.0  -46.2937  -54.01    0.0
 0.0  -62.3522  -65.769   0.0
 0.0  -54.898   -49.9944  0.0
 0.0  -61.0017  -44.7134  0.0
 0.0  -42.8678  -36.2393  0.0
 0.0  -57.5786  -48.2782  0.0
 0.0  -54.3562  -62.9777  0.0
 0.0  -47.5234  -43.2405  0.0
 0.0  -56.7916  -38.8615  0.0
 0.0  -43.445   -46.7103  0.0
 0.0  -53.8189  -62.0427  0.0
 0.0  -68.0537  -43.6257  0.0
 0.0  -28.0959  -55.0965  0.0
 0.0  -61.879   -51.7773  0.0

In [14]:
BpAlignGpu.update_g!(g, conditional, JP, tmpJP, tmpg, incg, elts_J)

In [15]:
g2 = BpAlignGpu.test_update_g(cond, J);

In [16]:
l=1
g2[:,:,l]

14×14 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 53.0462  53.9231  47.5018  54.5539  …  46.3816  53.1952  42.488   58.068
 66.4512  68.3391  60.2639  69.6767     58.6258  68.3295  53.1312  74.129
 51.3609  53.0764  48.0451  54.6595     46.0269  52.5608  41.8082  57.548
 48.5119  49.1684  44.1759  50.9492     42.3385  48.9692  38.7023  53.9851
 53.7732  54.3213  47.7367  55.9933     48.129   52.6751  43.0004  60.2441
 49.1237  51.364   45.012   51.0476  …  43.9971  51.1448  39.6926  54.9139
 43.2644  44.2695  40.1779  45.9844     38.3714  43.9317  34.4467  48.0369
 68.8303  70.4238  62.0447  71.0083     59.4006  69.7954  54.2202  75.0814
 57.5089  58.9469  51.6158  59.6037     49.5688  59.0284  45.4106  63.3844
 56.8501  57.7467  51.2116  59.3121     48.9344  56.8189  45.2318  62.3461
 49.965   51.7823  45.8019  52.6213  …  44.4192  51.4735  40.3714  56.2946
 44.8772  45.077   39.5947  46.1551     39.2307  44.408   35.3763  48.9697
 51.2663  51.9285  46.4352  53.3775     43.6292  51.4

In [17]:
g[:,:,l]

14×14 Matrix{Float32}:
 53.0462  53.9231  47.5018  54.5539  …  46.3816  53.1952  42.488   58.068
 66.4512  68.3391  60.2639  69.6767     58.6258  68.3295  53.1312  74.129
 51.3609  53.0765  48.0451  54.6595     46.0269  52.5608  41.8082  57.548
 48.5119  49.1684  44.1759  50.9492     42.3385  48.9692  38.7023  53.9851
 53.7732  54.3212  47.7367  55.9933     48.129   52.6751  43.0004  60.2441
 49.1238  51.364   45.012   51.0476  …  43.9971  51.1448  39.6926  54.9139
 43.2644  44.2695  40.1779  45.9844     38.3714  43.9317  34.4467  48.0369
 68.8303  70.4237  62.0446  71.0082     59.4006  69.7953  54.2202  75.0813
 57.5089  58.9469  51.6158  59.6037     49.5688  59.0284  45.4106  63.3844
 56.8501  57.7467  51.2116  59.3121     48.9344  56.8188  45.2318  62.3461
 49.965   51.7823  45.8019  52.6213  …  44.4193  51.4735  40.3714  56.2946
 44.8772  45.0769  39.5947  46.1551     39.2307  44.408   35.3763  48.9697
 51.2663  51.9285  46.4352  53.3775     43.6292  51.4381  40.3702  55.8179
 53.2